## Coclico exposed building

This workbench allows for visualizing the trend in the number of buildings affected by coastal flooding.This notebook requires the list of LAU_IDs, either a single value or multiple if selecting a larger unit, as well as the choice of return period.

Import

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import pystac_client
import geopandas as gpd









CoCliCo stack catalog (Links to the files produced by WP6)

In [ ]:
# load the CoCliCo STAC catalog
catalog = pystac_client.Client.open(
    "https://storage.googleapis.com/coclico-data-public/coclico/coclico-stac/catalog.json"
)
# catalog

# list the datasets present in the catalog, we are interested in the slp5 and slp6 sets
list(catalog.get_children())

In [ ]:
3
href = catalog.get_child("ssl").assets["data"].href#change for new data of vincent

Define parameter:
- study zone (LAU ID)
-scenario of coastal flood


In [ ]:

studyzone=[]#list of LAU ID
scenariocoastalflood="rp1ud"#

filter dataset

In [ ]:
def filter(df):
    build=df.loc[df['level_0'].isin(['building', 'healthcare', 'education'])]
    return df


creat a dataframe with just the data to draw

In [ ]:
def creatdf(df,studyzone,scenariocoastalflood):
    zonestudy=[]
    year=[2010,2030,2050,2100]
    scenario=['SSP1','SSP2','SSP5']
    data=[]
    namecolumns=["year","scenario","value"]
    for years in year:
        for scenarios in scenario:
            sum=0
            for zone in zonestudy:
                value = df.loc[zone, (scenariocoastalflood,scenarios, years,)]
                sum=sum+value
            data.append([years,scenario,sum])

    df = pd.DataFrame(data,columns=namecolumns)
    return df

In [ ]:
# data

#open file
df=gpd.read_parquet(href)
#df=open(....)

df=filter(df)
df=creatdf(df,studyzone,scenariocoastalflood)






# colors of scenario
colors_rgb = {
    'SSP1': (1.0, 0.0, 0.0),   # Rouge
    'SSP2': (0.0, 0.0, 1.0),   # Bleu
    'SSP5': (0.0, 1.0, 0.0),   # Vert
}


fig, ax = plt.subplots()

# for each scenario draw each point
for scenario in df['scenario'].unique():
    subset = df[df['scenario'] == scenario]
    ax.scatter(subset['year'], subset['value'], color=colors_rgb[scenario], label=scenario)

# information on axe x
ax.set_xticks([2010, 2030, 2050, 2100])
ax.set_xlim(2000, 2100)
ax.set_xlabel('Time horizon')

# information on axe x
ax.set_ylabel('number of buildings')

# display legend
ax.legend()

# display graph
plt.title('Number of Buildings over Time')
plt.grid(True)  # Afficher la grille pour mieux visualiser les points
plt.show()
